<a href="https://colab.research.google.com/drive/1xTRIszuDqxGcSzHPGhFRTSIGK2w4a6u6#scrollTo=pH7jQQasOrPg" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Limpieza de datos

Vamos a aplicar el [checklist del Banco Mundial](https://dimewiki.worldbank.org/wiki/Checklist:_Data_Cleaning) en Python usando principalmente pandas.

## 0. Prerequisitos

Aclaración: La próxima celda es para compatilibidad con Colab, NO ES RECOMENDADO realizar pip install desde un notebook.

In [ ]:
!pip install 'ftfy<5.6'
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# 1. Importando los datos

# 1.1. Verificar que no hay problemas en la importación

In [ ]:
import pandas as pd
pd.options.display.float_format = '{:.2f}'.format

In [ ]:
# sharing link: "https://drive.google.com/file/d/1QT8At_qS_BNaR5IW5cNhB0MQYSghqmvw/view?usp=sharing"

fileDownloaded = drive.CreateFile({'id':'1QT8At_qS_BNaR5IW5cNhB0MQYSghqmvw'})
fileDownloaded.GetContentFile('ks-projects-201801.csv')

Veamos de importar datos de proyectos de Kickstarter la plataforma de Crowdsourcing

In [ ]:
kickstarter_2018 = pd.read_csv('ks-projects-201801.csv')

Veamos los datos cargados en el dataframe

In [ ]:
kickstarter_2018

Por defecto solo vemos los valores al comienzo o al final del archivo.

Tomemos una muestra al azar para ver valores más dispersos

In [ ]:
# fijamos una semilla para reproducibilidad 
import numpy as np
np.random.seed(0)

In [ ]:
kickstarter_2018.sample(5)

No se observa a simple vista ningún problema obvio.

Notar que todos vimos los mismos resultados. Al fijar la semilla no hubo tal azar, esto es algo necesario cuando queremos "reproducir valores aleatorios"

Veamos la descripción del dataset si se corresponde con lo levantado https://www.kaggle.com/kemical/kickstarter-projects/data

In [ ]:
pd.DataFrame([["ID", "No description provided", "Numeric"],
["name", "No description provided", "String"],
["category", "No description provided", "String"],
["main_category", "No description provided", "String"],
["currency", "No description provided", "String"],
["deadline", "No description provided", "DateTime"],
["goal", "Goal amount in project currency", "Numeric"],
["launched", "No description provided", "DateTime"],
["pledged", "Pledged amount in the project currency", "Numeric"],
["state", "No description provided", "String"],
["backers", "No description provided", "Numeric"],
["country", "No description provided", "String"],
["usd pledged", "Pledged amount in USD (conversion made by KS)", "Numeric"],
["usd_pledged_real", "Pledged amount in USD (conversion made by fixer.io api)", "Numeric"],
["usd_goal_real", "Goal amount in USD", "Numeric"]], columns=["Field name","Field description", "Type"])

Ahora veamos los tipos de datos que detectó pandas

In [ ]:
kickstarter_2018.dtypes

Los campos object generalmente son String, entonces parece que no reconoció como fechas en **deadline** y **launched**

Veamos los datos un resumen de los datos

In [ ]:
kickstarter_2018.describe()

Por defecto se ven los datos numéricos, veamos el resto.

In [ ]:
kickstarter_2018.describe(include=['object'])

Operemos un cacho sobre los datos de lanzamiento

In [ ]:
kickstarter_2018['launched'].min()

Parece funcionar, pero ahora calculemos el rango de fechas de los proyectos

In [ ]:
kickstarter_2018['launched'].max() - kickstarter_2018['launched'].min()

Indiquemos que columnas son fechas como indica la [documentación](https://pandas.pydata.org/pandas-docs/stable/io.html#datetime-handling)

In [ ]:
kickstarter_2018 = pd.read_csv("ks-projects-201801.csv", parse_dates=["deadline","launched"])
kickstarter_2018.dtypes

Ahora vemos que esas columnas fueron reconocidas como fechas

Veamos la misma muestra de nuevo

In [ ]:
kickstarter_2018.sample(5)

Y veamos el resumen de los datos

In [ ]:
kickstarter_2018.describe(include='all')

Podemos ver que tenemos primero y último en el resumen de las columnas de fechas.

Ahora deberíamos poder calcular el rango de fechas de lanzamietos

In [ ]:
kickstarter_2018['launched'].max() - kickstarter_2018['launched'].min()

# 1.2. Asegurar de tener ids/claves únicas

Chequear que no hay datos duplicados

In [ ]:
kickstarter_2018.shape

Pandas soporta índices en los DataFrames vamos a recargar el conjunto de datos

In [ ]:
kickstarter_2018 = pd.read_csv("ks-projects-201801.csv", 
                               parse_dates=["deadline","launched"],
                               index_col=['ID'])

In [ ]:
kickstarter_2018

In [ ]:
kickstarter_2018.shape

De esta forma podemos buscar por el índice

In [ ]:
kickstarter_2018.loc[999988282]

También podemos verificar si hay filas de contenidos duplicado con la función `.duplicated()`

In [ ]:
kickstarter_2018[kickstarter_2018.duplicated()]

Como Pandas acepta valores duplicados en los índices también debemos verificar ahí

In [ ]:
pd.Series(kickstarter_2018.index,dtype=str).describe()

In [ ]:
kickstarter_2018[kickstarter_2018.index.duplicated()]

Repasamos con un ejemplo de juguete, creamos un string separado por comas y en varias lineas para combertirlo luego en tabla

In [ ]:
csv='1,2,A,2022\n3,3,B,2022\n1,3,B,2022\n1,3,A,2021'
print(csv)

In [ ]:
from io import StringIO
df = pd.read_csv(StringIO(csv), names=['id','value','name','year'], index_col='id')
df

Mostremos filas duplicadas

In [ ]:
df[df.duplicated()]

Mostremos filas con la variable name duplicada

In [ ]:
df[df.name.duplicated()]

In [ ]:
df[df.name.duplicated(keep=False)]

Seleccionemos todas las filas con índicies duplicadas


In [ ]:
df[df.index.duplicated(keep=False)]

#### Ejercicio 1:

Armar una tabla con todos los proyectos con nombres duplicados, ordenados para revisar agrupados. 

# 1.3. Despersonalizar datos y guardarlos en un nuevo archivo

Hay muchas técnicas para despersonalizar datos.

Para ilustrar mostramos las ofrecidas por Google https://cloud.google.com/dlp/docs/transformations-reference:

* **Reemplazo**: Reemplaza cada valor de entrada con un valor determinado.
* **Ocultamiento**: Quita un valor y lo oculta.
* **Enmascaramiento con caracteres**: Enmascara una string por completo o parcialmente mediante el reemplazo de un número determinado de caracteres con un carácter fijo especificado..
* **Seudonimización mediante el reemplazo de un valor de entrada con un hash criptográfico**: Reemplaza valores de entrada con una string hexadecimal de 32 bytes mediante una clave de encriptación de datos.
* **Cambio de fechas**: Cambia las fechas por un número de días al azar, con la opción de ser coherente en el mismo contexto..
* **Seudonimización mediante el reemplazo con token de preservación de formato criptográfico**: Reemplaza un valor de entrada con un token, o valor sustituto, de la misma longitud mediante la encriptación de preservación de formato (FPE) con el modo de operación FFX. Esto permite que se use el resultado en sistemas con validación de formato o que necesitan aparecer como reales a pesar de que la información no se revela.
* **Valores de depósito con base en rangos de tamaño fijos**: Enmascara los valores de entrada y los reemplaza por depósitos, o rangos dentro de los cuales se encuentra el valor de entrada.
* **Valores de depósito con base en rangos de tamaño personalizados**: Valores de entrada de depósito con base en rangos configurables por el usuario y valores de reemplazo.
* **Extracción de datos de tiempo**: EExtrae o preserva una porción de los valores Date, Timestamp y TimeOfDay.

In [ ]:
from hashlib import md5

Apliquemos la función MD5 hash para reemplazar los valores de nombres, veamos como aplicarlos sobre un string

Funciones asociadas a md5:


*   encode():  para convertir string a bytes
*   digest():  para retornar la información en bytes
*   hexdigest():  para recuperar la información en formato hexadecimal 

In [ ]:
string = 'Universidad Nacional de Córdoba'
md5(string)

Esto nos devuelve un error porque no se puede aplicar directamente a strings, para eso usamos la función `.encode()` para convertir los strings en bytes 

In [ ]:
md5(string.encode())

In [ ]:
md5(string.encode()).hexdigest()

Veamos de aplicarlo ahora a toda la columna de la variable nombre y veamos que tenemos

In [ ]:
def hashit(val):
    return md5(val.encode('utf-8'))

kickstarter_2018['name'].apply(hashit)

Esto nos da error al utilizar enconde con algún valor considerado como numérico, a continuación se puede tratar de ver donde es que hay problemas, queda de ejercicio entender que hace la siguiente cell

In [ ]:
def hashit(val):
    try:
        return md5(val.encode('utf-8'))
    except Exception as e:
        print(val, type(val))
        raise(e)

kickstarter_2018['name'].apply(hashit)

Finalmente una manera de aplicar hash a todos los nombre es la siguiente, convirtiendo cualqueir nombre al tipo string

In [ ]:
def hashit(val):
    if isinstance(val, float): 
        return str(val)
    return md5(val.encode('utf-8')).hexdigest()


kickstarter_2018['name'].apply(hashit)

#### Ejercicio 2:

Verificar que los proyectos que tienen nombres duplicados también tienen el hash de nombre duplicado

# 1.4. Nunca modificar los datos crudos u originales


A continuación guardamos el dataset con el que vamos a seguir trabajando en un nuevo archivo para no modificar el dataset original

In [ ]:
kickstarter_2018.to_csv("ks-projects-201801-mod.csv")


# 2. Pasos necesarios


In [ ]:
#import pandas as pd
kickstarter_2018 = pd.read_csv("ks-projects-201801-mod.csv",
                        index_col='ID',
                        parse_dates=['deadline','launched'])

In [ ]:
kickstarter_2018.describe(include='all')

## 2.1. Etiquetas de variables/columnas: no usar caracteres especiales



In [ ]:
# helpful character encoding module
import chardet

¿Por qué? Por que aun hay limitaciones para trabajar con estos caracteres.

¿Cúales son los caracteres "normales"? 

In [ ]:
import string

string.ascii_letters + string.digits

¿Qué es ASCII? Es una de las primeras codificaciones (o encoding) disponibles. Ver https://en.wikipedia.org/wiki/ASCII

Un encoding es un mapa de caracteres a una representación en bits (por ejemplo 1000001). ASCII es uno de los primeros estandares para interoperatividad entre computadoras (antes cada fabricante de computadora usa su propia codificación), contempla sólo los caracteres ingleses y usa 7 bits, por ejemplo *1000001* codifica el caracter *A*

Como sólo representaban los caracteres ingleses, empiezan a aparecer variantes para distintos idiomas, Microsoft hizo los propios para internacionalizar Windows, hubo otro estandar ISO 8859, hasta que llegó el éstandar Unicode que hizo múltiples encodings pero cada uno contempla múltiples idiomas e incluso es extensible a futuro. UTF-8 es de esos encodings el más utilizado. Ver https://en.wikipedia.org/wiki/Unicode

En particular, Python 3 utiliza UTF-8 por defecto (lo que no sucedía en Python 2).

Veamos un ejemplo práctico

In [ ]:
# start with a string
before = "This is the euro symbol: €"

# check to see what datatype it is
type(before)

In [ ]:
# encode it to a different encoding, replacing characters that raise errors
after = before.encode("utf-8", errors = "replace")

# check the type
type(after)

In [ ]:
# take a look at what the bytes look like
after

In [ ]:
# convert it back to utf-8
print(after.decode("utf-8"))

In [ ]:
# try to decode our bytes with the ascii encoding
print(after.decode("ascii"))

In [ ]:
# start with a string
before = "This is the euro symbol: €"

# encode it to a different encoding, replacing characters that raise errors
after = before.encode("ascii", errors = "replace")

# convert it back to utf-8
print(after.decode("ascii"))

# We've lost the original underlying byte string! It's been 
# replaced with the underlying byte string for the unknown character :(

Carguemos ahora otro dataset, con los proyectos a diciembre de 2016 para ver los detalles de los nombres 

In [ ]:
# sharing link: "https://drive.google.com/file/d/1eQVshDQy6xt4ghA0kp87Uyp3eoeK7LSL/view?usp=sharing"

fileDownloaded = drive.CreateFile({'id':'1eQVshDQy6xt4ghA0kp87Uyp3eoeK7LSL'})
fileDownloaded.GetContentFile('ks-projects-201612.csv')

In [ ]:
kickstarter_2016 = pd.read_csv("ks-projects-201612.csv")

El error nos indica que el archivo no estaría en Unicode. Si no está en algún metadato o no fue indicado es dificil detectar el encoding, por suerte hay un paquete para esto en Python, que aplica unas heurísticas para detectar encoding.

In [ ]:
# look at the first ten thousand bytes to guess the character encoding
with open("ks-projects-201612.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))

# check what the character encoding might be
result

Entonces le podemos indicar el encoding al leer el archivos

In [ ]:
# read in the file with the encoding detected by chardet
kickstarter_2016 = pd.read_csv("ks-projects-201612.csv", encoding='Windows-1252')

# look at the first few lines
kickstarter_2016.head()

Hay otro problema con los caracteres que es más sutíl: **[Mojibake](https://en.wikipedia.org/wiki/Mojibake)**

Sucede al leer UTF-8 como ASCII extendido (ie. `Windows-*`, `iso-8859-*` entre otros)

Ejemplo de mojibake:

In [ ]:
"cigüeña".encode('utf-8').decode('iso-8859-1')

Para esto vamos a usar un módulo específico [ftfy](https://ftfy.readthedocs.io/en/latest/). 

Que nos permite ordenar cadenas por rareza

In [ ]:
import ftfy.badness as bad

In [ ]:
def weird(val):
    if isinstance(val, float): 
        return 0
    return bad.sequence_weirdness(val)


kickstarter_2018['name_weirdness'] = kickstarter_2018['name'].apply(weird)
kickstarter_2018[kickstarter_2018['name_weirdness'] > 1]

Volviendo a la consigna original, chequeamos que no haya caracteres fuera de a-Z, 0-9 y _ en los nombres de columnas

In [ ]:
kickstarter_2018.columns[~kickstarter_2018.columns.str.match(r'^(\w+)$')]

In [ ]:
kickstarter_2018.usd_pledged.head()

In [ ]:
kickstarter_2018.columns = kickstarter_2018.columns.str.replace(' ', '_')
kickstarter_2018.head()

In [ ]:
kickstarter_2018.usd_pledged.head()

#### Ejercicio 3

Comparar la cantidad de nombres raros en kickstarter_2018 con la que obtenemos al cargar 'ks-projects-201801.csv' con encoding iso-8859-1.

## 2.2. Tratar valores faltantes

Veamos cuantos valores nulos tenemos

In [ ]:
# get the number of missing data points per column
missing_values_count = kickstarter_2018.isnull().sum()

# look at the # of missing points in the first ten columns
missing_values_count[missing_values_count > 0]

Esta medida en sí, no dice nada más que con cuantos valores debemos lidiar. 

Primero tenemos que ver si en términos relativos su impacto.

In [ ]:
len(kickstarter_2018.dropna())/len(kickstarter_2018)

In [ ]:
len(kickstarter_2018.dropna(subset=['name']))/len(kickstarter_2018)

También debemos ver que significan en términos de representatividad e importancia de nuestro análisis.

In [ ]:
kickstarter_2018[kickstarter_2018.name.isnull()]

Ejemplo, cuantos de los proyectos suspendidos no tienen nombre

In [ ]:
len(kickstarter_2018[kickstarter_2018.state == 'suspended'].dropna(subset=['name'])) / \
    len(kickstarter_2018[kickstarter_2018.state == 'suspended'])

En este caso, creemos que es seguro descartar los proyectos sin nombre

In [ ]:
kickstarter_2018 = kickstarter_2018.dropna(subset=['name'])

In [ ]:
missing_values_count = kickstarter_2018.isnull().sum()
missing_values_count[missing_values_count > 0]

In [ ]:
kickstarter_2018[kickstarter_2018.usd_pledged.isnull()]

In [ ]:
kickstarter_2018[kickstarter_2018.usd_pledged.isnull()].describe(include='all')

Interesante, todos los datos pertenecen a un país indeterminado.

Todos los datos de ese "país" no tienen dinero prometido?

In [ ]:
kickstarter_2018[(kickstarter_2018.country == 'N,0"') & ~(kickstarter_2018.usd_pledged.isnull())].head()

A priori como los proyectos no tienen patrocinantes, completamos en 0 los valores

In [ ]:
kickstarter_2018 = kickstarter_2018.fillna(0)
missing_values_count = kickstarter_2018.isnull().sum()
missing_values_count[missing_values_count > 0]

In [ ]:
kickstarter_2018.shape

Más métodos disponibles en https://pandas.pydata.org/pandas-docs/stable/api.html#api-dataframe-missing y http://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing

## 2.3. Codificar variables

Para trabajar con los algoritmos de aprendizaje automático, las variables categóricas estas deben ser codificadas como variables numéricas, no como cadenas.

Para esta tarea también hay diferentes estrategias, dos comunes son: asociar cadena a número y asociar cadena a columna.

In [ ]:
from sklearn import preprocessing

In [ ]:
kickstarter_2018.describe(include=['O'])

In [ ]:
column='main_category'
# Create a label (category) encoder object
le = preprocessing.LabelEncoder()
# Fit the encoder to the pandas column
le.fit(kickstarter_2018[column])

In [ ]:
# View encoder mapping
dict(zip(le.classes_,le.transform(le.classes_)))

In [ ]:
# Apply the fitted encoder to the pandas column
kickstarter_2018[column] = le.transform(kickstarter_2018[column]) 
kickstarter_2018.head()

In [ ]:
# Reversing encoding
kickstarter_2018[column] = le.inverse_transform(kickstarter_2018[column]) 
kickstarter_2018.head()

In [ ]:
from sklearn.preprocessing import LabelBinarizer
other_column = 'state'

lb = LabelBinarizer()

lb_results = lb.fit_transform(kickstarter_2018[other_column])
pd.DataFrame(lb_results, columns=((other_column + '_') + pd.Series(lb.classes_))).head(10)

More about preprocessing in http://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing

#### Ejercicio 4

Codificar `currency` con ambas estrategias

## 2.4. No cambiar los nombres de las variables de la fuente de origen


## 2.5. Verificar la consistencia de las variables
Aplicar reglas de integridad


In [ ]:
kickstarter_2018.head()

¿Hay proyectos sin patrocinadores pero con plata prometida?

In [ ]:
kickstarter_2018[(kickstarter_2018.backers == 0) & (kickstarter_2018.usd_pledged > 0)]

¿Hay proyecto no exitosos que pasaron el objetivo?

In [ ]:
kickstarter_2018[(kickstarter_2018.state != 'successful') & (kickstarter_2018.pledged > kickstarter_2018.goal)]

¿Cómo es, en promedio, la correlación entre el objetivo y la plata obtenida?

In [ ]:
df = kickstarter_2018.groupby('state')[['goal','pledged']].aggregate('mean')
df['pledged'] / df['goal']

¿Cuánta plata se obtiene en promedio por estado y categoría?

In [ ]:
kickstarter_2018.pivot_table(values='usd_pledged',index='state',columns='main_category')

#### Ejercicio 5

1. ¿Hay proyecto éxitosos que no consiguieron el objetivo? Si hay, ¿Qué porcentaje sí y cuál no?
2. Calcular una tabla con la cantidad de proyectos por categoría principal y estado.

## 2.6. Identificar y documentar valores atípicos/outliers


Queremos analizar las características de los proyectos que obtienen dinero pero sin considerar los casos atípicos.

In [ ]:
%matplotlib inline
kickstarter_2018.usd_pledged.hist();

In [ ]:
kickstarter_2018.boxplot(column='usd_pledged');

"There are many rules of thumb for how to define an outlier but there is no silver bullet. One rule of thumb is that any data point that is three standard deviations away from the mean of the same data point for all observations."

In [ ]:
outliers = kickstarter_2018[kickstarter_2018.usd_pledged > (kickstarter_2018.usd_pledged.mean() + 
                                                            3 * kickstarter_2018.usd_pledged.std())]
outliers

In [ ]:
kickstarter_2018.drop(outliers.index).boxplot(column='usd_pledged');

In [ ]:
kickstarter_2018.drop(outliers.index).boxplot(column='usd_pledged',by='state');

Analizar outliers por categorías

In [ ]:
kickstarter_2018.boxplot(column='usd_pledged',by='state');
kickstarter_2018.boxplot(column='usd_goal_real',by='state');

#### Ejercicio 6

Calcular los valores atípicos de 'usd_goal_real' y graficar los boxplots, con y sin estos valores por categoría

## 2.7. Evaluar cómo comprimir los datos 
para su almacenamiento más eficiente.

En la actualidad, en las nubes el almacenamiento no suele ser problema sino la velocidad para leerlo que suele ser determinante a la hora de determinar el formato de almacenamiento.


## 2.8. Guardar el set de datos con un nombre informativo.


EL problema de la ingeniería de software